In [ ]:
from ReadLoad import read_jsonl, write_jsonl, read_json, write_json
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core import Document
from tqdm import tqdm

test_dataB = read_jsonl("dataset/test-B-0722.json")

In [ ]:
documents = [Document(text=t['条款'],
                      metadata={
                          '产品名': t['产品名'],
                          'ID': t['ID'],
                          '问题': t['问题']
                      }) for t in test_dataB]
for data in tqdm(documents):
    data.text = data.text.replace('。','. ')
    data.text = data.text.replace('！','! ')
    data.text = data.text.replace('？','? ')
    
# documents[:5]

In [6]:
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core import Document

parser = SentenceWindowNodeParser.from_defaults(
    # how many sentences on either side to capture
    window_size=3,
    # the metadata key that holds the window of surrounding sentences
    window_metadata_key="window",
    # the metadata key that holds the original sentence
    original_text_metadata_key="original_sentence"
)
nodes = parser.get_nodes_from_documents(documents[:5])


In [63]:
import nltk


# 下载punkt数据包
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data] Error downloading 'punkt' from
[nltk_data]     <https://raw.githubusercontent.com/nltk/nltk_data/gh-
[nltk_data]     pages/packages/tokenizers/punkt.zip>:   <urlopen error
[nltk_data]     Tunnel connection failed: 400 Bad Request>


False

In [64]:
import os
from dotenv import dotenv_values
from llama_index.legacy.llms import OpenAILike as OpenAI
# from llama_index.core import Settings
# config = dotenv_values(".env")

#     # 初始化 LLM 嵌入模型 和 Reranker
# llm = OpenAI(
#     model_name="qwen2-7b-instruct",
#     api_key=config['qwen_key'],
#     api_base=config['qwen_url'],
#     is_chat_model=True
# )
#Settings.llm = llm

BadZipFile: File is not a zip file

In [ ]:
from modelscope import snapshot_download
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

model_dir = snapshot_download('Xorbits/bge-small-zh-v1.5')
embeding = HuggingFaceEmbedding(
        model_name=model_dir,
        cache_folder="./",
        embed_batch_size=512,
    )
Settings.embed_model = embeding

In [10]:
from llama_index.core import VectorStoreIndex
index = VectorStoreIndex(nodes)

In [ ]:
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.vector_stores import (
    MetadataFilter,
    MetadataFilters,
    FilterOperator,
)
query = test_dataB[1]['问题']
filters = MetadataFilters(
    filters=[
        MetadataFilter(
            key="问题", value=query
        ),
    ]
)
vector_retriever = index.as_retriever(similarity_top_k=2,filters=filters)
retrieve_nodes = vector_retriever.retrieve(query)
print(query)
for node in retrieve_nodes:
    print(node.id_)
    print(node.text)
    print(node.metadata['window'])
    print(node.score)
    print("=====")

In [37]:
from llama_index.core import PromptTemplate

new_text_qa_template_str = (
    """\
    上下文信息如下：
    ----------
    {context_str}
    ----------
    根据上下文信息而非先验知识，构建一个经过严谨思考且内容详实的答案，来回答问题。
    充分运用上下文信息来支撑你的答案，并确保回答符合人类的偏好以及遵循指示的原则。
    如果上下文信息没有相关知识，可以回答不确定，不要复述上下文信息。
    
    问题：{query_str}
    回答：\
    """
)

In [32]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores import ExactMatchFilter, MetadataFilters,MetadataFilter
text_qa_template = PromptTemplate(new_text_qa_template_str)
query_engine = RetrieverQueryEngine.from_args(
        retriever = vector_retriever, 
        llm=glm,
        text_qa_template=text_qa_template,
        #refine_template=refine_template,
        response_mode="compact",
        node_postprocessors=[reranker],
        )

'如需要发票，请前往【众安保险官网-保险服务-保单查询】查询保单并下载电子发票；\n理赔服务\n1.医疗、津贴责任支持在线理赔，您可以在【众安保险官方营业厅】，发起理赔申请，并提供完整申请资料（初审通过后，部分案件仍需寄送材料原件）.  （路径： APP—品牌专区—进入【众安保险官方营业厅】找到【我的服务】—【在线理赔】选中保单，进行在线理赔）\n2.身故、伤残等保险责任，出险后拨打客服电话952299或1010-9955进行报案.  具体电话理赔流程如下：\n1）拨打众安客服热线952299或1010-9955报案并获取索赔指引；\n2）准备证明保险事故的相关材料；\n3）理赔材料原件寄送至：上海市黄浦区北京东路130号中实大楼5楼，数字生活理赔部收，联系电话：952299或10109955，邮编：200002；\n4）确属保险责任范围，众安保险在收到齐全理赔材料后10天内理赔，疑难复杂案件最长不超过30个自然日；\n5）赔款资金将支付至被保险人账户或受益人账户\n众安客服热线\n1010-9955，众安保险投诉热线：02180399188，众安保险官网：www.zhongan.com. \n 【重要告知】\n1、投保时，本人已就该产品的保障内容以及保险金额向被保险人进行了明确说明，并征得其同意. \n 2、确认购买本保险产品前，请您仔细阅读并充分理解并同意接受本《投保须知》和《保险条款》等，特别是其中有关责任条款及免除责任条款、退保规则、投保人和被保险人义务的内容等重大事项. \n 3、同时请您了解，在投保本产品前您应履行相应的如实告知义务，具体如下：投保人或被保险人应如实填写投保信息，并就保险公司提出的询问据实告知，否则保险公司有权根据《中华人民共和国保险法》第十六条的规定解除保险合同且不承担赔偿责任；订立保险合同时，保险公司就保险标的或者被保险人的有关情况提出询问的，投保人应当如实告知.  投保人故意或者因重大过失未履行前款规定的如实告知义务，足以影响保险公司决定是否同意承保或者提高保险费率的，保险公司有权解除合同. '